# Parameter Values

In this notebook, we explain how parameter values are set for a model. Information on how to add parameter values is provided in our [online documentation](https://pybamm.readthedocs.io/en/latest/tutorials/add-parameter-values.html)

## Setting up parameter values

In [1]:
import pybamm
import tests
import numpy as np
import os
import matplotlib.pyplot as plt
from pprint import pprint
os.chdir(pybamm.__path__[0]+'/..')

In `pybamm`, the object that sets parameter values for a model is the `ParameterValues` class, which extends `dict`. This takes the values of the parameters as input, which can be either a dictionary,

In [2]:
param_dict = {"a": 1, "b": 2, "c": 3}
parameter_values = pybamm.ParameterValues(param_dict)
print("parameter values are {}".format(parameter_values))

parameter values are <pybamm.parameters.parameter_values.ParameterValues object at 0x7f1f70335080>


or a csv file,

In [3]:
f = open("input/param_file.csv", "w+")
f.write(
"""
Name [units],Value
a, 4
b, 5
c, 6
"""
)
f.close()

parameter_values = pybamm.ParameterValues("input/param_file.csv")
print("parameter values are {}".format(parameter_values))

parameter values are <pybamm.parameters.parameter_values.ParameterValues object at 0x7f1f70335518>


or using one of the pre-set chemistries

In [4]:
print("Marquis2019 chemistry set is {}".format(pybamm.parameter_sets.Marquis2019))
chem_parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Marquis2019)
print("Negative current collector thickness is {} m".format(
    chem_parameter_values["Negative current collector thickness [m]"])
)

Marquis2019 chemistry set is {'chemistry': 'lithium-ion', 'cell': 'kokam_Marquis2019', 'anode': 'graphite_mcmb2528_Marquis2019', 'separator': 'separator_Marquis2019', 'cathode': 'lico2_Marquis2019', 'electrolyte': 'lipf6_Marquis2019', 'experiment': '1C_discharge_from_full_Marquis2019'}
Negative current collector thickness is 2.5e-05 m


We can input functions into the parameter values, either directly (note we bypass the check that the parameter already exists)

In [5]:
def cubed(x):
    return x ** 3
parameter_values.update({"cube function": cubed}, check_already_exists=False)
print("parameter values are {}".format(parameter_values))

parameter values are <pybamm.parameters.parameter_values.ParameterValues object at 0x7f1f70335518>


or by using `pybamm.load_function` to load from a path to the function or just a name (in which case the whole directory is searched)

In [6]:
f = open("input/squared.py","w+")
f.write(
"""
def squared(x):
    return x ** 2
"""
)
f.close()
parameter_values.update({"square function": pybamm.load_function("squared.py")}, check_already_exists=False)
print("parameter values are {}".format(parameter_values))

parameter values are <pybamm.parameters.parameter_values.ParameterValues object at 0x7f1f70335518>


## Setting parameters for an expression

We represent parameters in models using the classes `Parameter` and `FunctionParameter`. These cannot be evaluated directly,

In [7]:
a = pybamm.Parameter("a")
b = pybamm.Parameter("b")
c = pybamm.Parameter("c")
func = pybamm.FunctionParameter("square function", a)

expr = a + b * c
try:
    expr.evaluate()
except NotImplementedError as e:
    print(e)

method self.evaluate() not implemented
               for symbol a of type <class 'pybamm.expression_tree.parameter.Parameter'>


However, the `ParameterValues` class can walk through an expression, changing an `Parameter` objects it sees to the appropriate `Scalar` and any `FunctionParameter` object to the appropriate `Function`, and the resulting expression can be evaluated

In [8]:
expr_eval = parameter_values.process_symbol(expr)
print("{} = {}".format(expr_eval, expr_eval.evaluate()))

a + b * c = 34.0


In [9]:
func_eval = parameter_values.process_symbol(func)
print("{} = {}".format(func_eval, func_eval.evaluate()))

16.0 = 16.0


If a parameter needs to be changed often (for example, for convergence studies or parameter estimation), the `InputParameter` class should be used. This is not fixed by parameter values, and its value can be set on evaluation (or on solve):

In [10]:
d = pybamm.InputParameter("d")
expr = 2 + d
expr_eval = parameter_values.process_symbol(expr)
print("with d = {}, {} = {}".format(3, expr_eval, expr_eval.evaluate(u={"d": 3})))
print("with d = {}, {} = {}".format(5, expr_eval, expr_eval.evaluate(u={"d": 5})))

with d = 3, 2.0 + d = 5.0
with d = 5, 2.0 + d = 7.0


## Solving a model

The code below shows the entire workflow of:

1. Proposing a toy model
2. Discretising and solving it first with one set of parameters,
3. then updating the parameters and solving again

The toy model used is:
$$\frac{\mathrm{d} u}{\mathrm{d} t} = -a u$$

with initial conditions $u(0) = b$. The model is first solved with $a = 3, b = 2$, then with $a = 4, b = -1$

In [11]:
# Create model
model = pybamm.BaseModel()
u = pybamm.Variable("u")
a = pybamm.Parameter("a")
b = pybamm.Parameter("b")
model.rhs = {u: -a * u}
model.initial_conditions = {u: b}
model.variables = {"u": u}

# Set parameters, with b as an input ########################
parameter_values = pybamm.ParameterValues({"a": 3, "b": "[input]"})
parameter_values.process_model(model)
#############################################################

# Discretise using default discretisation
disc = pybamm.Discretisation()
disc.process_model(model)

# Solve
t_eval = np.linspace(0, 2, 30)
ode_solver = pybamm.ScipySolver()
solution = ode_solver.solve(model, t_eval, inputs={"b": 2})

# Post-process, so that u1 can be called at any time t (using interpolation)
t_sol1 = solution.t
u1 = solution["u"]

# Solve again with different inputs ###############################
solution = ode_solver.solve(model, t_eval, inputs={"b": -1})
t_sol2 = solution.t
u2 = solution["u"]
###################################################################

# Plot
t_fine = np.linspace(0,t_eval[-1],1000)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13,4))
ax1.plot(t_fine, 2 * np.exp(-3 * t_fine), t_sol1, u1(t_sol1), "o")
ax1.set_xlabel("t")
ax1.legend([" * exp(-3 * t)", "u1"], loc="best")
ax1.set_title("a = 3, b = 2")

ax2.plot(t_fine, - np.exp(-3 * t_fine), t_sol2, u2(t_sol2), "o")
ax2.set_xlabel("t")
ax2.legend(["-exp(-3 * t)", "u2"], loc="best")
ax2.set_title("a = 3, b = -1")


plt.tight_layout()
plt.show()

KeyError: "Input parameter 'b' not found"

In [ ]:
model.rhs

## Printing parameter values

In most models, it is useful to define dimensionless parameters, which are combinations of other parameters. However, since parameters objects must be processed by the `ParameterValues` class before they can be evaluated, it can be difficult to quickly check the value of a dimensionless parameter. 

You can print all of the dimensionless parameters in a model by using the `print_parameters` function. Note that the `print_parameters` function also gives the dependence of the parameters on C-rate (as some dimensionless parameters vary with C-rate), but we can ignore that here

In [ ]:
parameters = {"a": a, "b": b, "a + b": a + b, "a * b": a * b}
param_eval = pybamm.print_parameters(parameters, parameter_values)
for name, (value,C_dependence) in param_eval.items():
    print("{}: {}".format(name, value))

If you provide an output file to `print_parameters`, the parameters will be printed to that output file.